<a href="https://colab.research.google.com/github/graphtrek/stockforecast/blob/main/graphtrek_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Test
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import requests
import json
import os, time

from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow import keras
tf.__version__

'2.6.0'

In [3]:
ticker = "ZM"
look_back = 12
stock_api_response = pd.read_json('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+ticker+'&outputsize=full&apikey=3F4URDEKOPLFH25T')

In [14]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('Stock Portfolio')
rows = worksheet.get_worksheet(3).get_all_values()
print(rows)



# Convert to a DataFrame and render.
import pandas as pd
stocks_df = pd.DataFrame.from_records(rows)
try :
  nr_of_shares = stocks_df.loc[stocks_df[1] == ticker][8].values[0]
  all_costs = stocks_df.loc[stocks_df[1] == ticker][9].values[0]
  cost_per_share = stocks_df.loc[stocks_df[1] == ticker][10].values[0]
except:
  nr_of_shares = 0
  all_costs = 0
  cost_per_share = 0
  
print('Shares:', nr_of_shares,'Cost:',all_costs,'Cost Per Share:',cost_per_share)

[['Investment Category', 'Stock Ticker', 'Google Price', 'Manual Price', 'Last Price', 'Last DPS', 'Yield on Cost', 'Last Price Yield', 'Shares', 'Cost', 'Cost (Per Share)', 'Unrealized Gain/Loss', 'Unrealized Gain/Loss (%)', 'Realized Gain/Loss', 'Dividends Collected', 'Total Gain/Loss', 'Mkt Value'], ['Dividend', 'HOG', '37.22', '0.000', '37.22', '0.000', '0.00%', '0.00%', '10.0', '$397.90', '$39.79', '-$25.70', '-6.5%', '$0.00', '$0.00', '-$25.70', '$372.20'], ['Dividend', 'LVS', '38.22', '0.000', '38.22', '0.000', '0.00%', '0.00%', '20.0', '$853.25', '$42.66', '-$88.85', '-10.4%', '$0.00', '$0.00', '-$88.85', '$764.40'], ['Dividend', 'CAT', '195.16', '0.000', '195.16', '0.000', '0.00%', '0.00%', '5.0', '$996.90', '$199.38', '-$21.10', '-2.1%', '$0.00', '$0.00', '-$21.10', '$975.80'], ['Dividend', 'DIS', '176.74', '0.000', '176.74', '0.000', '0.00%', '0.00%', '6.0', '$1,059.44', '$176.57', '$1.00', '0.1%', '$0.00', '$0.00', '$1.00', '$1,060.44'], ['Dividend', 'CCL', '23.83', '0.000'

In [ ]:
stock_api_data = stock_api_response['Time Series (Daily)']
#print(stock_api_data)
stock_api_data = stock_api_data.drop(index=['1. Information','2. Symbol','3. Last Refreshed','4. Output Size','5. Time Zone']);
#print(list(stock_api_data.items()))
data = []
for key, value in stock_api_data.items():
  data.append([
      pd.to_datetime(key,).date(),
      value.get('1. open'),
      value.get('2. high'),
      value.get('3. low'),
      value.get('5. adjusted close'),
      value.get('6. volume')
      ])

last_data =  str(data[0][0])
print(data[0])
data = np.flip(data[:1000],axis=0)


In [ ]:
model = None
model_file_path = '/content/drive/MyDrive/models/'+ticker+'.h5'
try:
  model = keras.models.load_model(model_file_path)
  modified = os.path.getmtime(model_file_path)

  print(data[0][0] - datetime.fromtimestamp(modified).date() )
  print('Loaded', ticker , 'model train date:',datetime.fromtimestamp(modified).date() , 'last data:', last_data)
except:
  model = None
  print('Model ' + ticker + ' does not exists.')

In [ ]:
df = pd.DataFrame(data,columns=['Date','Open','High','Low','Close','Volume'])

df['50MA'] = df['Close'].rolling(50).mean()
df['100MA'] = df['Close'].rolling(100).mean()
df['200MA'] = df['Close'].rolling(200).mean()

split_percent = 0.90
split = int(split_percent*len(data))

split

In [ ]:
close_data = df['Close'].values
close_data = np.asarray(close_data).astype(np.float32)
close_data = close_data.reshape((-1))

train_data = close_data[:split]
test_data = close_data[split:]

train_dates = df['Date'][:split]
test_dates = df['Date'][split:]

volumes = np.asarray(df['Volume'].values).astype(np.int)[-150:]
volume_dates = df['Date'][-150:]
#len(test_data), test_data

In [ ]:
trace1 = go.Scatter(
    x = train_dates,
    y = train_data,
    mode = 'lines',
    line=dict(width=3),
    name = 'Train'
)

trace2 = go.Scatter(
    x = test_dates,
    y = test_data,
    mode='lines',
    line=dict(width=3),
    name ='Test'
)

trace3 = go.Scatter(
    x = df['Date'],
    y = df['50MA'],
    mode='lines',
    name ='50MA'
)

trace4 = go.Scatter(
    x = df['Date'],
    y = df['100MA'],
    mode='lines',
    name ='100MA'
)

trace5 = go.Scatter(
    x = df['Date'],
    y = df['200MA'],
    mode='lines',
    name ='200MA'
)

layout = go.Layout(
    title = ticker + ' Date:' + last_data + ' Open:' + str(data[-1][1]) + ' High:' + str(data[-1][2]) + ' Low:' + str(data[-1][3]) + ' Close:' + str(data[-1][4]),
    xaxis = {'title' : "Dates"},
    yaxis = {'title' : "Close Price ($)"},
    height = 450
)
fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5], layout=layout)
fig.show()


fig = px.area(x=volume_dates, y=volumes, height=250, labels={
                     "x": "Dates",
                     "y": "Volume"
                 },
                title=ticker)
fig.show()

In [ ]:
# Normalizing data, scale between 0 and 1:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = sc.fit_transform(train_data.reshape((-1,1)))
test_data_scaled = sc.fit_transform(test_data.reshape((-1,1)))
train_data_scaled.shape, test_data_scaled.shape

In [ ]:
if model is None:
  # Building Model:
  # The LSTM layer expects the number of time steps and the number of features to work properly.
  model = tf.keras.models.Sequential()

  model.add(tf.keras.layers.LSTM(units=50, return_sequences=True, activation="tanh",
                                input_shape=(look_back, 1)))

  model.add(tf.keras.layers.Dropout(0.2))

  model.add(tf.keras.layers.LSTM(units=50, return_sequences=True, activation="tanh"))

  model.add(tf.keras.layers.Dropout(0.2))

  model.add(tf.keras.layers.LSTM(units=50, return_sequences=True, activation="tanh"))

  model.add(tf.keras.layers.Dropout(0.2))

  model.add(tf.keras.layers.LSTM(units=50, activation="tanh"))

  model.add(tf.keras.layers.Dropout(0.2))

  model.add(tf.keras.layers.Dense(units=1))

  model.compile(
      loss=tf.keras.losses.MAE,
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
      metrics=["mae"]
  )
  
  print('Model ' + ticker + ' compiled.')

  train_generator = TimeseriesGenerator(train_data_scaled, train_data_scaled, length=look_back)     
  
  modelo = model.fit(train_generator, epochs=100, verbose=0)
  model.save(model_file_path)
  print('Saved model ' + ticker)

  plt.plot(modelo.history['loss'])
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.show()

In [ ]:
test_generator = TimeseriesGenerator(test_data_scaled, test_data_scaled, length=look_back)
predicted_stock_price_scaled = model.predict(test_generator)

In [ ]:
predicted_stock_price = sc.inverse_transform(predicted_stock_price_scaled)
zoom = 700
trace1 = go.Scatter(
    x = train_dates[zoom:],
    y = train_data[zoom:],
    mode = 'lines',
    name = 'Train'
)
trace2 = go.Scatter(
    x = test_dates,
    y = test_data,
    mode='lines',
    name = 'Test'
)
trace3 = go.Scatter(
    x = test_dates,
    y = predicted_stock_price.reshape((-1)),
    mode='lines',
    line=dict(width=3),
    name = 'Predict'
)
layout = go.Layout(
    title = ticker + ' Date:' + last_data + ' Open:' + str(data[-1][1]) + ' High:' + str(data[-1][2]) + ' Low:' + str(data[-1][3]) + ' Close:' + str(data[-1][4]),
    xaxis = {'title' : "Dates"},
    yaxis = {'title' : "Close Price ($)"},
    height = 600
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()
fig.write_html('/content/drive/MyDrive/models/'+ticker+ '_' + last_data + '_predict.html')

In [ ]:
def predict(num_prediction, model):
    prediction_list = test_data_scaled[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = look_back
forecast_scaled = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

forecast = sc.inverse_transform(forecast_scaled.reshape((-1,1)))
#forecast_dates,forecast

In [ ]:
zoom = 850

trace1 = go.Candlestick(
    x = train_dates[zoom:],
    open = df['Open'].values[zoom:],
    high = df['High'].values[zoom:],
    low = df['Low'].values[zoom:],
    close = train_data[zoom:],
#    mode = 'lines',
#    line=dict(width=3),
    name = 'Train'
)
trace2 = go.Candlestick(
    x = test_dates,
    open = df['Open'].values[split:],
    high = df['High'].values[split:],
    low = df['Low'].values[split:],
    close = test_data,
#    mode='lines',
#    line=dict(width=3),
    name = 'Test'
)
trace3 = go.Scatter(
    x = test_dates,
    y = predicted_stock_price.reshape((-1)),
    mode='lines',
    name = 'Predict'
)
trace4 = go.Scatter(
    x = forecast_dates,
    y = forecast.reshape((-1)),
    mode='lines',
    line=dict(width=3),
    name = 'Forecast'
)
trace5 = go.Scatter(
    x = df['Date'][zoom:],
    y = df['50MA'][zoom:],
    mode='lines',
    name ='50MA'
)
trace6 = go.Scatter(
    x = df['Date'][zoom:],
    y = df['100MA'][zoom:],
    mode='lines',
    name ='100MA'
)
trace7 = go.Scatter(
    x = df['Date'][zoom:],
    y = df['200MA'][zoom:],
    mode='lines',
    name ='200MA'
)


print('Shares:', nr_of_shares,'Cost:',all_costs,'Cost Per Share:', cost_per_share, '50MA:', int(df['50MA'].values[-1]))
print('Forecast:',forecast.reshape((-1)).astype(np.int))
layout = go.Layout(
    title = 
    '<b>'+ticker+ '</b>' + ' Date:' + last_data + '<BR>' 
    'Open:' + str(data[-1][1]) + 
    ' High:' + str(data[-1][2]) + 
    ' Low:' + str(data[-1][3]) + 
    ' Close:' + str(data[-1][4]) + 
    ' ' +
    'Forecast in ' + str(len(forecast_dates)) + ' days ' + str(np.around(forecast.mean())) + 
    '<BR>' + 
    'Shares:' + str(nr_of_shares) +
    ' Cost: ' + str(all_costs).replace('$','') +
    ' Cost Per Share: ' + str(cost_per_share).replace('$',''),
    xaxis = {'title' : 'Dates'},
    yaxis = {'title' : 'Close Price ($)'},
    height = 600,
    font=dict(
        family="Courier New, monospace",
        size=8,
        color="#7f7f7f"
    )
)


fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5, trace6, trace7], layout=layout)

fig.update_yaxes(showspikes=True, spikemode='across', spikesnap='cursor',spikedash='dash')
fig.update_xaxes(showspikes=True, spikemode='across', spikesnap='cursor', spikedash='dash')
fig.update_layout(xaxis_rangeslider_visible=False)



config = dict({'scrollZoom': True})
fig.show(config=config)

fig.write_html('/content/drive/MyDrive/models/'+ticker+ '_' + last_data + '_forecast.html')

fig = px.area(x=volume_dates, y=volumes, height=250, labels={
                     "x": "Dates",
                     "y": "Volume"
                 },
                title=ticker)
fig.show()